In [ ]:
import random

import boto3

session = boto3.Session(profile_name="boto3_101")

Using the aws `client`


In [7]:
rn = random.randrange(10**5, 10**6)

s3_client = session.client("s3")
s3_client.create_bucket(Bucket=f"boto3-bucket-{rn}")

{'ResponseMetadata': {'RequestId': 'QD44XXWY84RB3N8R',
  'HostId': 'E4kTCqBst4+VVrgdKBUREqXHcMtDP44ZeIfSGhnUzB4/Hiib5U2pCmyUUz4BSLZDI5qEiHmIqpU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'E4kTCqBst4+VVrgdKBUREqXHcMtDP44ZeIfSGhnUzB4/Hiib5U2pCmyUUz4BSLZDI5qEiHmIqpU=',
   'x-amz-request-id': 'QD44XXWY84RB3N8R',
   'date': 'Sun, 16 Mar 2025 20:54:08 GMT',
   'location': '/boto3-bucket-293993',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/boto3-bucket-293993'}

In [8]:
s3_client.delete_bucket(Bucket="boto3-bucket-293993")

{'ResponseMetadata': {'RequestId': 'P3V0EGMS852BCFG7',
  'HostId': 'iCkKaktBdxHAaBsWyrlz+iY4qF6c2Bjf6uk399D2cBaMA37ZefYDhA0GBRPYB+36hDwf/KrX6wM=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'iCkKaktBdxHAaBsWyrlz+iY4qF6c2Bjf6uk399D2cBaMA37ZefYDhA0GBRPYB+36hDwf/KrX6wM=',
   'x-amz-request-id': 'P3V0EGMS852BCFG7',
   'date': 'Sun, 16 Mar 2025 21:00:40 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [ ]:
s3_client.list_buckets()["Buckets"]

[{'Name': 'add-demo-cloudfront-v1',
  'CreationDate': datetime.datetime(2024, 6, 18, 12, 47, 1, tzinfo=tzutc())},
 {'Name': 'add-demo-s3-v1',
  'CreationDate': datetime.datetime(2024, 6, 14, 16, 26, 57, tzinfo=tzutc())},
 {'Name': 'sagemaker-studio-891377239273-vees33vfkeb',
  'CreationDate': datetime.datetime(2024, 10, 25, 5, 35, 20, tzinfo=tzutc())},
 {'Name': 'sagemaker-us-east-1-891377239273',
  'CreationDate': datetime.datetime(2024, 10, 25, 5, 35, 25, tzinfo=tzutc())},
 {'Name': 'terraform-backend-frz9g',
  'CreationDate': datetime.datetime(2024, 9, 11, 9, 35, 37, tzinfo=tzutc())}]

In [137]:
list_s3_buckets = lambda: [
    bucket["Name"] for bucket in s3_client.list_buckets()["Buckets"]
]
list_s3_buckets()

['add-demo-cloudfront-v1',
 'add-demo-s3-v1',
 'sagemaker-studio-891377239273-vees33vfkeb',
 'sagemaker-us-east-1-891377239273',
 'terraform-backend-frz9g']

In [ ]:
def bucket_name_generator(prefix: str = "boto3-bucket", random_digits: int = 6) -> str:
    """
    Generate random names for aws s3 buckets in format: "prefix-random_digits",
    then create state that holds bucket names generated via `.names` attr.
    """
    rn = random.randrange(10 ** (random_digits - 1), 10 ** (random_digits))
    bucket_name = f"{prefix}-{rn}"

    # create state that holds bucket names generated
    if not hasattr(bucket_name_generator, "names"):
        bucket_name_generator.names = []
    bucket_name_generator.names.append(bucket_name)

    return bucket_name

In [164]:
def create_buckets(num: int, prefix: str, random_digits: int) -> None:
    """
    Create aws s3 buckets with in format "prefix-random_digits" in `num` number of times.
    """
    for _ in range(num):
        s3_client.create_bucket(
            Bucket=bucket_name_generator(prefix=prefix, random_digits=random_digits)
        )

In [165]:
create_buckets(3, "boto3-bucket", 4)

In [166]:
bucket_name_generator.names

['boto3-bucket-9138',
 'boto3-bucket-6308',
 'boto3-bucket-3762',
 'boto3-bucket-2952',
 'boto3-bucket-8246',
 'boto3-bucket-9457',
 'boto3-bucket-3277',
 'boto3-bucket-2090',
 'boto3-bucket-4502']

In [ ]:
def delete_buckets(bucket_names: list, contains_name: str | None = None) -> list:
    """
    Delete aws s3 buckets from generated bucket names and/or buckets containing the name in `contains_name` param.

    Also cleans the `bucket_names` container after bucket deletion.

    Returns list of current aws s3 buckets
    """

    # delete buckets from `bucket_names` seq
    for _, bucket_name in enumerate(bucket_names):
        s3_client.delete_bucket(Bucket=bucket_name)

    s3_buckets = lambda: [
        bucket["Name"] for bucket in s3_client.list_buckets()["Buckets"]
    ]

    # delete buckets using `contains_name` value
    if contains_name:
        for bucket_name in s3_buckets():
            if contains_name in bucket_name:
                s3_client.delete_bucket(Bucket=bucket_name)

    bucket_names.clear()

    return s3_buckets()

In [170]:
list_s3_buckets()

['add-demo-cloudfront-v1',
 'add-demo-s3-v1',
 'sagemaker-studio-891377239273-vees33vfkeb',
 'sagemaker-us-east-1-891377239273',
 'terraform-backend-frz9g']

In [171]:
delete_buckets(bucket_name_generator.names)

['add-demo-cloudfront-v1',
 'add-demo-s3-v1',
 'sagemaker-studio-891377239273-vees33vfkeb',
 'sagemaker-us-east-1-891377239273',
 'terraform-backend-frz9g']

In [172]:
bucket_name_generator.names

[]

Using the aws `resource`
